In [16]:
%reload_ext autoreload
%autoreload 2
import sys
from dotenv import load_dotenv
import os
import pandas as pd
import torch
from transformers import BertTokenizer
load_dotenv()

ROOT = os.getenv("ROOT")
sys.path.append(ROOT)

from src.nlp.embedder import EmbedderPipeline
from src.data_loader import load_reviews
from src.processing import reviews_processing

In [17]:
df = load_reviews(category="All_beauty", frac=0.01)
df = reviews_processing(df=df, clean_text=False)

In [11]:
pipeline = EmbedderPipeline()

In [12]:
pipeline._opt_setup()

In [14]:
pipeline._train()

Epoch [1/5], Loss: 5.2004
Epoch [2/5], Loss: 0.7170
Epoch [3/5], Loss: 1.8543
Epoch [4/5], Loss: 0.8627
Epoch [5/5], Loss: 0.5901


In [18]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
max_len = 32

def get_embedding(text, model, tokenizer, max_len=32):
    model.eval()
    with torch.no_grad():
        inputs = tokenizer(text, padding="max_length", truncation=True, max_length=max_len, return_tensors="pt")
        input_ids = inputs["input_ids"]
        attention_mask = inputs["attention_mask"]
        embedding = model(input_ids, attention_mask)
        return embedding.cpu().numpy()  # convert embedding tensor to a numpy array

In [ ]:
df["embeddings"] = df["review_input"].apply(lambda text: get_embedding(text, pipeline.embedding_net, tokenizer, max_len))